In [ ]:
from moccasin import setup_notebook

setup_notebook()


In [ ]:
from moccasin.config import get_active_network

active_netwok = get_active_network()
print(f"Active network: {active_netwok.name}")

In [ ]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6)
def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc, weth):
    # print(f"Starting balance of WETH: {weth.balanceOf(boa.env.eoa)}")
    print(f"USDC balance before {usdc.balanceOf(boa.env.eoa)}")
    weth.deposit(value=STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print(f"USDC balance after: {usdc.balanceOf(boa.env.eoa)}")
    # print(f"Ending balance of WETH: {weth.balanceOf(boa.env.eoa)}")

def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract]:
    print("Starting setup script...")

    # 1. Give ourselves some ETH
    # 2. Give ourselves some USDC and WETH
    active_network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)
        

def moccasin_main():
    setup_script()

moccasin_main()

In [ ]:
active_network = get_active_network()

usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [ ]:
usdc.balanceOf(boa.env.eoa)
weth.balanceOf(boa.env.eoa)

In [ ]:
from moccasin.config  import get_or_initialize_config

config = get_or_initialize_config()
config.reload()
active_network = config.get_active_network()
aavev3_pool_address_provider = active_network.manifest_named("aavev3_pool_address_provider")
pool_address = aavev3_pool_address_provider.getPool()
print(f"Aave V3 Pool address: {pool_address}")

config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address =pool_address)

In [ ]:
REFERAL_CODE = 0

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)

    print(f"Depositing {token.address} into Aave contract {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERAL_CODE)


usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)


(
    totalCollateralBase,
    totalDebtBase,
    availableBorrowsBase,
    currentLiquidationThreshold,
    ltv,
    healthFactor,
) = pool_contract.getUserAccountData(boa.env.eoa)

print(f"""User account data:
totalCollateralBase: {totalCollateralBase}
totalDebtBase: {totalDebtBase}
availableBorrowsBase: {availableBorrowsBase}
currentLiquidationThreshold: {currentLiquidationThreshold}
ltv: {ltv}
healthFactor: {healthFactor}
""")


In [ ]:
# 30% USDC and 70% WETH

config.reload()
active_network = config.get_active_network()
aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

In [ ]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("weth", address=a_token[1])

    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address=a_token[1])

print(a_usdc)
print(a_weth)

        

In [ ]:
# Get how much they are worth, to figure out what our portfolio makeup is 
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) #6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa) #18 decimals

a_usdc_balance_normalized = a_usdc_balance / (1_000_000)
a_weth_balance_normalized = a_weth_balance / (1_000_000_000_000_000_000)

print(a_usdc_balance_normalized)
print(a_weth_balance_normalized)

In [ ]:
def get_price(feed_name: str) -> float:
    active_network = config.get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    decimals_normalized = (10 ** decimals)
    return price / decimals_normalized

usdc_price = get_price("usdc_usd")
weth_price = get_price("eth_usd")
print(f"USDC price: {usdc_price}")
print(f"ETH price: {weth_price}")    

In [ ]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price
total_value = usdc_value + weth_value

target_usdc_value = 0.3
target_weth_value = 0.7

usdc_percent_allocation = usdc_value / total_value
weth_percent_allocation = weth_value / total_value

BUFFER = 0.1

needs_rebalancing = (
    abs(usdc_percent_allocation - target_usdc_value) > BUFFER or
    abs(weth_percent_allocation - target_weth_value) > BUFFER
)
print(needs_rebalancing)
print(usdc_percent_allocation)
print(weth_percent_allocation)

In [ ]:
a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)

def print_token_balance():
    print(f"USDC balance: {usdc.balanceOf(boa.env.eoa)}")   
    print(f"WETH balance: {weth.balanceOf(boa.env.eoa)}")
    print(f"aUSDC balance: {a_usdc.balanceOf(boa.env.eoa)}")
    print(f"aWETH balance: {a_weth.balanceOf(boa.env.eoa)}")

print_token_balance()



In [ ]:
usdc_data = {"balance": a_usdc_balance_normalized, "price": usdc_price, "contract": a_usdc}
weth_data = {"balance": a_weth_balance_normalized, "price": weth_price, "contract": a_weth}
target_allocations = {"usdc": target_usdc_value, "weth": target_weth_value}

def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }

trades = calculate_rebalancing_trades(usdc_data, weth_data, target_allocations)
print(trades)

weth_to_sell = trades["weth"]["trade"]

In [ ]:
# # Reload config & network
# config.reload()
# active_network = config.get_active_network()

# # Load Uniswap V3 Swap Router (must be saved already)
# uniswap_swap_router = active_network.manifest_named("uniswap_v3_swap_router")

# # 1. Read REAL on-chain WETH balance
# weth_balance = weth.balanceOf(boa.env.eoa)

# # 2. Decide how much to sell (sell 50% for safety)
# amount_weth = weth_balance // 2

# # 3. Safety check (VERY IMPORTANT)
# assert amount_weth > 0, "Not enough WETH to swap"

# # 4. Approve Uniswap router
# weth.approve(uniswap_swap_router.address, amount_weth)

# print("Swapping WETH → USDC...")

# # 5. Swap WETH → USDC
# uniswap_swap_router.exactInputSingle(
#     {
#         "tokenIn": weth.address,
#         "tokenOut": usdc.address,
#         "fee": 3000,                  # 0.3% pool
#         "recipient": boa.env.eoa,
#         "amountIn": amount_weth,
#         "amountOutMinimum": 0,        # ⚠️ OK ONLY for testing
#         "sqrtPriceLimitX96": 0,
#     },
#     simulate=False
# )

# # 6. Print balances
# def print_token_balance():
#     print(f"WETH balance : {weth.balanceOf(boa.env.eoa)}")
#     print(f"USDC balance : {usdc.balanceOf(boa.env.eoa)}")

# print_token_balance()

config.reload()
active_network = config.get_active_network()
uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")

amount_weth = abs(int(weth_to_sell * (10 ** 18)))

weth.approve(uniswap_swap_router.address, amount_weth)
min_out = int((trades["usdc"]["trade"] * (10 ** 6)) * 0.90)
print("Let's swap!")
    # struct ExactInputSingleParams {
    #     address tokenIn;
    #     address tokenOut;
    #     uint24 fee;
    #     address recipient;
    #     uint256 amountIn;
    #     uint256 amountOutMinimum;
    #     uint160 sqrtPriceLimitX96;
    # }
uniswap_swap_router.exactInputSingle(
    (
        weth.address,
        usdc.address,
        3000,
        boa.env.eoa,
        amount_weth,
        min_out,
        0
    )
)
print_token_balance()

In [ ]:
amount = usdc.balanceOf(boa.env.eoa)
deposit(pool_contract, usdc, amount)

print_token_balance()

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) 
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_normalized = a_usdc_balance / (1_000_000)
a_weth_balance_normalized = a_weth_balance / (1_000_000_000_000_000_000)

usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price

usdc_percent_allocation = usdc_value / (usdc_value + weth_value)
weth_percent_allocation = weth_value / (usdc_value + weth_value)


print(f"New USDC allocation: {usdc_percent_allocation}")
print(f"New WETH allocation: {weth_percent_allocation}")

In [ ]:
amount = weth.balanceOf(boa.env.eoa)
deposit(pool_contract, weth, amount)


In [ ]:
print_token_balance()

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) 
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_normalized = a_usdc_balance / (1_000_000)
a_weth_balance_normalized = a_weth_balance / (1_000_000_000_000_000_000)

usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price

weth_percent_allocation = weth_value / (usdc_value + weth_value)
usdc_percent_allocation = usdc_value / (usdc_value + weth_value)


print(f"New USDC allocation: {usdc_percent_allocation}")
print(f"New WETH allocation: {weth_percent_allocation}")